In [31]:
import pandas as pd
import numpy as np
import functools as ft
import itertools as it
import multiprocessing as mp
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [3]:
df = pd.read_csv("../data/amigo_grab1a.txt", sep='\t', low_memory=False, header=None);

In [5]:
col_names = ['Source', 'Internal ID', 'Gene/Product', 'Annotation Qualifier', 'Evidence', 'Ontology', 'Gene/Product Name','Date','Contributor','Annotation Extension', 'GO Class', 'GO Class Label', 'GO Class (Direct)', 'GO Class (Direct) Label', 'PANTHER Family', 'PANTHER Family Label']

In [6]:
len(col_names)

16

Looks like the GO Class Label is an aggregate of the GO Class (Direct) Label (e.g. the direct has 1 observation while the GO class has all observations)

In [8]:
df.columns = col_names
print(df.shape)
df.head()

(33000, 16)


,Source,Internal ID,Gene/Product,Annotation Qualifier,Evidence,Ontology,Name,Date,Contributor,Annotation Extension,GO Class,GO Class Label,GO Class (Direct),GO Class (Direct) Label,PANTHER Family,PANTHER Family Label
0,UniProtKB,P25788,PSMA3,NaN,IMP,P,Proteasome subunit alpha type-3,20160119,UniProt,NaN,GO:0019538|GO:0043603|GO:1901564|GO:0051171|GO...,protein metabolic process|cellular amide metab...,GO:0052548,regulation of endopeptidase activity,PANTHER:PTHR11599,proteasome subunit alpha/beta pthr11599
1,UniProtKB,P25786,PSMA1,NaN,IEA,P,Proteasome subunit alpha type-1,20180714,Ensembl,NaN,GO:0080134|GO:0048519|GO:0002682|GO:0002683|GO...,regulation of response to stress|negative regu...,GO:0002862,negative regulation of inflammatory response t...,PANTHER:PTHR11599,proteasome subunit alpha/beta pthr11599
2,UniProtKB,Q9UI17,DMGDH,NaN,NAS,P,"Dimethylglycine dehydrogenase, mitochondrial",20081204,UniProt,NaN,GO:0065007|GO:0008150|GO:0065008|GO:0008152|GO...,biological regulation|biological_process|regul...,GO:0019695,choline metabolic process,PANTHER:PTHR13847,fad nad binding oxidoreductases pthr13847
3,UniProtKB,Q9UI17,DMGDH,NaN,IMP,P,"Dimethylglycine dehydrogenase, mitochondrial",20170614,UniProt,NaN,GO:0009310|GO:0009056|GO:1901565|GO:0042133|GO...,amine catabolic process|catabolic process|orga...,GO:0042426,choline catabolic process,PANTHER:PTHR13847,fad nad binding oxidoreductases pthr13847
4,UniProtKB,Q9UI17,DMGDH,NaN,TAS,P,"Dimethylglycine dehydrogenase, mitochondrial",20180523,Reactome,NaN,GO:0009310|GO:0009056|GO:1901565|GO:0042133|GO...,amine catabolic process|catabolic process|orga...,GO:0042426,choline catabolic process,PANTHER:PTHR13847,fad nad binding oxidoreductases pthr13847


In [91]:
df[df['Gene/Product'] == 'PSMA3']['GO Class Label'][0] == 'protein metabolic process|cellular amide metabolic process|organonitrogen compound metabolic process|regulation of nitrogen compound metabolic process|primary metabolic process|cellular metabolic process|molecular_function|macromolecule metabolic process|regulation of protein metabolic process|regulation of cellular process|catalytic activity|peptide metabolic process|regulation of catalytic activity|regulation of cellular amide metabolic process|regulation of metabolic process|cellular nitrogen compound metabolic process|biological regulation|biological_process|regulation of macromolecule metabolic process|catalytic activity, acting on a protein|regulation of proteolysis|metabolic process|endopeptidase activity|peptidase activity|regulation of molecular function|cellular process|nitrogen compound metabolic process|peptidase activity, acting on L-amino acid peptides|proteolysis|regulation of endopeptidase activity|regulation of biological process|organic substance metabolic process|regulation of primary metabolic process|regulation of cellular metabolic process|regulation of hydrolase activity|hydrolase activity|regulation of peptidase activity'

True

In [56]:
df_2 = pd.read_csv("../data/amigo_grab1b.txt", sep='\t', low_memory=False, header=None);
df_3 = pd.read_csv("../data/amigo_grab2.txt", sep='\t', low_memory=False, header=None);

df_2.columns =  col_names
df_3.columns = col_names

#labels_1 = []

#dat = pd.concat([df, df_2, df_3])

In [57]:
for i in known_kinases[0:5]:
    print(i in df['Gene/Product'] or i in df_2['Gene/Product'] or i in df_3['Gene/Product'])

False
False
False
False
False


In [86]:
print(list(filter(lambda x: 'UFO' in x, df_3['Name']))) #.apply(lambda x: x.lower())

['Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO', 'Tyrosine-protein kinase receptor UFO']

In [85]:
'ufo' in list(map(lambda x: x.lower(), known_kinases))

True

In [87]:
df_3[df_3['Name'] == 'Tyrosine-protein kinase receptor UFO']

,Source,Internal ID,Gene/Product,Annotation Qualifier,Evidence,Ontology,Name,Date,Contributor,Annotation Extension,GO Class,GO Class Label,GO Class (Direct),GO Class (Direct) Label,PANTHER Family,PANTHER Family Label
49205,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0008150|GO:0040011|GO:0007275|GO:0048856|GO...,biological_process|locomotion|multicellular or...,GO:0001764,neuron migration,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49206,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:1903131|GO:0008150|GO:0007275|GO:0030097|GO...,mononuclear cell differentiation|biological_pr...,GO:0001779,natural killer cell differentiation,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49207,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0048771|GO:0008150|GO:0001974|GO:0032501,tissue remodeling|biological_process|blood ves...,GO:0001974,blood vessel remodeling,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49208,UniProtKB,P30530,AXL,NaN,IDA,P,Tyrosine-protein kinase receptor UFO,20120308,UniProt,NaN,GO:0008150|GO:0016192|GO:0051179|GO:0051234|GO...,biological_process|vesicle-mediated transport|...,GO:0006909,phagocytosis,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49209,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0008150|GO:0006954|GO:0006952|GO:0006950|GO...,biological_process|inflammatory response|defen...,GO:0006954,inflammatory response,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49210,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0048232|GO:0008150|GO:0044703|GO:0048609|GO...,male gamete generation|biological_process|mult...,GO:0007283,spermatogenesis,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49211,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,GOC,NaN,GO:0018108|GO:0043603|GO:0019538|GO:0036211|GO...,peptidyl-tyrosine phosphorylation|cellular ami...,GO:0018108,peptidyl-tyrosine phosphorylation,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49212,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,GOC,NaN,GO:0018108|GO:0043603|GO:0019538|GO:0036211|GO...,peptidyl-tyrosine phosphorylation|cellular ami...,GO:0018108,peptidyl-tyrosine phosphorylation,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49213,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0008150|GO:0060322|GO:0040011|GO:0007275|GO...,biological_process|head development|locomotion...,GO:0021885,forebrain cell migration,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
49214,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0008150|GO:0048731|GO:0007275|GO:0048856|GO...,biological_process|system development|multicel...,GO:0031100,animal organ regeneration,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416


In [83]:
df_2[df_2['Name'] == 'Tyrosine-protein kinase receptor UFO']

,Source,Internal ID,Gene/Product,Annotation Qualifier,Evidence,Ontology,Name,Date,Contributor,Annotation Extension,GO Class,GO Class Label,GO Class (Direct),GO Class (Direct) Label,PANTHER Family,PANTHER Family Label
4841,UniProtKB,P30530,AXL,NaN,IDA,P,Tyrosine-protein kinase receptor UFO,20120307,UniProt,CL:0000837,GO:0034097|GO:0060760|GO:0007154|GO:0048518|GO...,response to cytokine|positive regulation of re...,GO:0001961,positive regulation of cytokine-mediated signa...,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4842,UniProtKB,P30530,AXL,NaN,TAS,P,Tyrosine-protein kinase receptor UFO,20030904,PINC,NaN,GO:0065007|GO:0008150|GO:0050794|GO:0007165|GO...,biological regulation|biological_process|regul...,GO:0007165,signal transduction,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4843,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0065007|GO:0008150|GO:0065008|GO:0050817|GO...,biological regulation|biological_process|regul...,GO:0030168,platelet activation,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4844,UniProtKB,P30530,AXL,NaN,IDA,P,Tyrosine-protein kinase receptor UFO,20120307,UniProt,CL:0000837,GO:0065007|GO:0008150|GO:0048519|GO:0032649|GO...,biological regulation|biological_process|negat...,GO:0032689,negative regulation of interferon-gamma produc...,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4845,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0065007|GO:0008150|GO:0071706|GO:0032680|GO...,biological regulation|biological_process|tumor...,GO:0032720,negative regulation of tumor necrosis factor p...,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4846,UniProtKB,P30530,AXL,NaN,IDA,P,Tyrosine-protein kinase receptor UFO,20120307,UniProt,CL:0000837,GO:0048513|GO:0051094|GO:0002682|GO:0048518|GO...,animal organ development|positive regulation o...,GO:0032825,positive regulation of natural killer cell dif...,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4847,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0065007|OBA:0000001|GO:0008150|GO:0065008|G...,biological regulation|biological attribute|bio...,GO:0034101,erythrocyte homeostasis,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4848,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0065007|GO:0008150|GO:0007165|GO:0007154|GO...,biological regulation|biological_process|signa...,GO:0043491,protein kinase B signaling,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4849,UniProtKB,P30530,AXL,NaN,IEA,P,Tyrosine-protein kinase receptor UFO,20180714,Ensembl,NaN,GO:0065007|GO:0008150|GO:0060548|GO:0043067|GO...,biological regulation|biological_process|negat...,GO:0043524,negative regulation of neuron apoptotic process,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416
4850,UniProtKB,P30530,AXL,NaN,TAS,P,Tyrosine-protein kinase receptor UFO,20180523,Reactome,NaN,GO:0065007|GO:0008150|GO:0007165|GO:0007154|GO...,biological regulation|biological_process|signa...,GO:0048010,vascular endothelial growth factor receptor si...,PANTHER:PTHR24416,tyrosine-protein kinase receptor pthr24416


In [59]:
df[df['Name'] == 'Bone morphogenetic protein receptor type-1B']

,Source,Internal ID,Gene/Product,Annotation Qualifier,Evidence,Ontology,Name,Date,Contributor,Annotation Extension,GO Class,GO Class Label,GO Class (Direct),GO Class (Direct) Label,PANTHER Family,PANTHER Family Label
2489,UniProtKB,O00238,BMPR1B,NaN,IMP,P,Bone morphogenetic protein receptor type-1B,20080613,BHF-UCL,NaN,GO:0007275|GO:0048856|GO:0048513|GO:0051094|GO...,multicellular organism development|anatomical ...,GO:0030501,positive regulation of bone mineralization,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2490,UniProtKB,O00238,BMPR1B,NaN,NAS,P,Bone morphogenetic protein receptor type-1B,20041103,UniProt,NaN,GO:0070848|GO:0006366|GO:0007178|GO:0051171|GO...,response to growth factor|transcription by RNA...,GO:0030509,BMP signaling pathway,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2491,UniProtKB,O00238,BMPR1B,NaN,IDA,P,Bone morphogenetic protein receptor type-1B,20080613,BHF-UCL,NaN,GO:0070848|GO:0006366|GO:0007178|GO:0051171|GO...,response to growth factor|transcription by RNA...,GO:0030509,BMP signaling pathway,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2492,UniProtKB,O00238,BMPR1B,NaN,TAS,P,Bone morphogenetic protein receptor type-1B,20180523,Reactome,NaN,GO:0070848|GO:0006366|GO:0007178|GO:0051171|GO...,response to growth factor|transcription by RNA...,GO:0030509,BMP signaling pathway,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2493,UniProtKB,O00238,BMPR1B,NaN,ISS,P,Bone morphogenetic protein receptor type-1B,20160122,UniProt,NaN,GO:0061035|GO:0061036|GO:0007275|GO:0032330|GO...,regulation of cartilage development|positive r...,GO:0032332,positive regulation of chondrocyte differentia...,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2494,UniProtKB,O00238,BMPR1B,NaN,IMP,P,Bone morphogenetic protein receptor type-1B,20041103,UniProt,NaN,GO:0065007|GO:0008150|GO:0045597|GO:0051094|GO...,biological regulation|biological_process|posit...,GO:0045597,positive regulation of cell differentiation,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2495,UniProtKB,O00238,BMPR1B,NaN,IMP,P,Bone morphogenetic protein receptor type-1B,20080613,BHF-UCL,NaN,GO:0065007|GO:0008150|GO:0045597|GO:0051094|GO...,biological regulation|biological_process|posit...,GO:0045669,positive regulation of osteoblast differentiation,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2496,UniProtKB,O00238,BMPR1B,NaN,IMP,P,Bone morphogenetic protein receptor type-1B,20150724,BHF-UCL,UniProtKB:P81172|UBERON:0002107|GO:0071773,GO:0051173|GO:0006366|GO:0051171|GO:0044238|GO...,positive regulation of nitrogen compound metab...,GO:0045944,positive regulation of transcription by RNA po...,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2497,UniProtKB,O00238,BMPR1B,NaN,ISS,P,Bone morphogenetic protein receptor type-1B,20141217,AgBase,NaN,GO:0065007|GO:0008150|GO:0061035|GO:0061036|GO...,biological regulation|biological_process|regul...,GO:0061036,positive regulation of cartilage development,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...
2498,UniProtKB,O00238,BMPR1B,NaN,ISS,P,Bone morphogenetic protein receptor type-1B,20141217,AgBase,NaN,GO:0065007|GO:0008150|GO:0061035|GO:0061036|GO...,biological regulation|biological_process|regul...,GO:0061036,positive regulation of cartilage development,PANTHER:PTHR23255,transforming growth factor-beta receptor type ...


In [60]:
print('UFO' in df['Gene/Product'] or 'UFO' in df_2['Gene/Product'] or 'UFO' in df_3['Gene/Product'])

False


In [53]:
kinase_network_df = pd.read_csv('../data/Full_Kinome_Network_Compiled_no_header.txt', header=None, sep='\t')
known_kinases = list(set(kinase_network_df[0]) | set(kinase_network_df[1]))
print(known_kinases)

['EPHAA', 'WNK4', 'CLK4', 'MP2K5', 'RYK', 'UFO', 'KS6A1', 'FGFR3', 'BRSK1', 'KSR1', 'PHKG1', 'MTOR', 'STK36', 'IKKE', 'LMTK2', 'KCC2B', 'GAK', 'TAOK1', 'NTRK2', 'GSK3A', 'AAPK2', 'MAPK3', 'ARAF', 'PAK6', 'KSYK', 'EPHA3', 'PI3R4', 'KCRM', 'HIPK1', 'ERN1', 'PRPK', 'TRIO', 'KGP2', 'LATS2', 'ST38L', 'PIM2', 'CSK', 'STK3', 'ROR2', 'WNK3', 'ITK', 'KC1A', 'ILK', 'STK10', 'ATR', 'EPHB6', 'MP2K2', 'CDK1', 'STRAA', 'SGK2', 'VRK1', 'K6PP', 'SGK1', 'AURKC', 'DAPK2', 'ZAP70', 'M3K11', 'PI42B', 'ROR1', 'DYRK2', 'CDC7', 'MP2K3', 'BRD3', 'MYO3A', 'CSF1R', 'TRPM7', 'HXK2', 'CDKL5', 'DUSTY', 'PGFRA', 'KSR2', 'RIPK4', 'PI42A', 'DYRK3', 'LYN', 'CHK1', 'TYRO3', 'SIK3', 'KKCC1', 'MAPK5', 'KGP1', 'TNK1', 'TSSK2', 'KPCG', 'ERBB3', 'TIF1A', 'TYK2', 'KAD1', 'GUC2C', 'NRBP', 'ROS1', 'ARBK2', 'RIOK3', 'VRK3', 'KPCD', 'RIPK3', 'BMX', 'CDK14', 'MK13', 'TLK2', 'PDK1', 'M3K4', 'KAD2', 'VGFR2', 'PDK3', 'KT3K', 'CDK9', 'HUNK', 'AVR2A', 'TGFR2', 'KCC2D', 'MK03', 'PDK2', 'PIM1', 'HXK1', 'LTK', 'ACVR1', 'PMYT1', 'DAPK1', 

In [14]:
go_labels_tokenized = df.groupby('Gene/Product')['GO Class Label'].apply(lambda x: '|'.join(x)).apply(lambda x: x.split('|'))

In [45]:
print(len(set(go_labs)))
len(set(go_labs) | set(labs))

10204


10204

In [41]:
list_of_tokenized_labels = list(it.chain.from_iterable(go_labels_tokenized.values))
print(len(set(list_of_tokenized_labels)))

go_labs = Counter(list_of_tokenized_labels).keys()
go_freq = Counter(list_of_tokenized_labels).values()

[sorted(zip(list(go_freq), list(go_labs)), reverse=True)]

10204


[[(33000, 'biological_process'),
  (33000, 'biological regulation'),
  (29608, 'regulation of biological process'),
  (26758, 'cellular process'),
  (24748, 'regulation of cellular process'),
  (11222, 'response to stimulus'),
  (10950, 'positive regulation of biological process'),
  (10453, 'metabolic process'),
  (9965, 'signaling'),
  (9924, 'cell communication'),
  (9800, 'organic substance metabolic process'),
  (9741, 'cellular response to stimulus'),
  (9561, 'cellular metabolic process'),
  (9514, 'regulation of metabolic process'),
  (9193, 'macromolecule metabolic process'),
  (9154, 'signal transduction'),
  (9082, 'primary metabolic process'),
  (8729, 'nitrogen compound metabolic process'),
  (8656, 'cellular nitrogen compound metabolic process'),
  (8484, 'regulation of macromolecule metabolic process'),
  (8374, 'regulation of cellular metabolic process'),
  (8100, 'positive regulation of cellular process'),
  (7989, 'regulation of primary metabolic process'),
  (7713, '

In [16]:
labs = Counter(df['GO Class (Direct) Label']).keys()
freq = Counter(df['GO Class (Direct) Label']).values()

In [22]:
[sorted(zip(list(freq), list(labs)), reverse=True)]

[(784, 'positive regulation of transcription by RNA polymerase II'),
 (611, 'regulation of transcription by RNA polymerase II'),
 (601, 'G-protein coupled receptor signaling pathway'),
 (508, 'negative regulation of transcription by RNA polymerase II'),
 (491, 'signal transduction'),
 (435, 'positive regulation of transcription, DNA-templated'),
 (395, 'regulation of transcription, DNA-templated'),
 (365, 'negative regulation of transcription, DNA-templated'),
 (317, 'positive regulation of cell proliferation'),
 (285, 'negative regulation of apoptotic process'),
 (276, 'regulation of signaling receptor activity'),
 (233, 'positive regulation of gene expression'),
 (232, 'negative regulation of cell proliferation'),
 (215, 'positive regulation of apoptotic process'),
 (206, 'intracellular signal transduction'),
 (194, 'cytokine-mediated signaling pathway'),
 (184, 'fibroblast growth factor receptor signaling pathway'),
 (166, 'positive regulation of GTPase activity'),
 (151, 'positive 

In [ ]:
go_direct_labels_tokenized = df.groupby('Gene/Product')['GO Class (Direct) Label'].apply(lambda x: '|'.join(x)).apply(lambda x: x.split('|'))

In [11]:
df['GO Class Label']

0        [protein metabolic process, cellular amide met...
1        [regulation of response to stress, negative re...
2        [biological regulation, biological_process, re...
3        [amine catabolic process, catabolic process, o...
4        [amine catabolic process, catabolic process, o...
5        [biological regulation, protein-containing com...
6        [cellular amide metabolic process, protein met...
7        [protein modification process, positive regula...
8        [protein modification process, positive regula...
9        [biological regulation, biological_process, re...
10       [biological regulation, biological_process, sm...
11       [biological regulation, biological_process, re...
12       [biological regulation, biological_process, po...
13       [biological regulation, biological_process, tu...
14       [animal organ development, positive regulation...
15       [regulation of cell-cell adhesion, positive re...
16       [regulation of kinase activity, protein modifi.

In [ ]:
[{x:True for x in y} for y in ]

In [10]:
len(df['Gene/Product'].unique())

5842